In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from collections import deque
import threading
from playsound import playsound
import os

# === Load CNN Model ===
model = tf.keras.models.load_model(r"C:\Users\blatw\OneDrive\Desktop\drowsiness_model.h5")

# === Alarm Sound Path ===
alarm_path = r"C:\Users\blatw\OneDrive\Desktop\alarm.mp3.mp3"
if not os.path.exists(alarm_path):
    raise FileNotFoundError("❌ Alarm file not found!")

alarm_active = False

def play_alarm():
    global alarm_active
    playsound(alarm_path)
    alarm_active = False

# === EAR Calculation ===
def calculate_EAR(landmarks, eye_indices):
    eye = np.array([landmarks[i] for i in eye_indices])
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

# === Face Mesh Initialization ===
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [263, 387, 385, 362, 380, 373]

# === Webcam Initialization ===
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Webcam not detected.")
    exit()

prediction_window = deque(maxlen=30)
EAR_THRESHOLD = 0.21
FRAME_COUNT_THRESHOLD = 10  # Faster detection
frame_count = 0

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = face_mesh.process(rgb)
        EAR_detected = False
        EAR_value = 0
        label = "Alert"

        if results.multi_face_landmarks:
            landmarks = results.multi_face_landmarks[0].landmark
            h, w, _ = frame.shape
            coords = [(int(p.x * w), int(p.y * h)) for p in landmarks]

            left_EAR = calculate_EAR(coords, LEFT_EYE)
            right_EAR = calculate_EAR(coords, RIGHT_EYE)
            EAR_value = (left_EAR + right_EAR) / 2.0
            EAR_detected = True

            print(f"👁 EAR: {EAR_value:.3f}")

            if EAR_value < EAR_THRESHOLD:
                frame_count += 1
            else:
                frame_count = 0

            if frame_count >= FRAME_COUNT_THRESHOLD:
                small_frame = cv2.resize(frame, (64, 64))
                input_img = np.expand_dims(small_frame, axis=0) / 255.0
                pred = model.predict(input_img, verbose=0)[0][0]
                label = "Drowsy" if pred > 0.5 else "Alert"
                prediction_window.append(1 if label == "Drowsy" else 0)
                print(f"🤖 CNN Prediction: {pred:.2f} -> {label}")
            else:
                # Fallback if not enough frames, use EAR directly
                label = "Drowsy" if EAR_value < EAR_THRESHOLD else "Alert"
                prediction_window.append(1 if label == "Drowsy" else 0)
        else:
            prediction_window.append(0)

        avg_drowsy = sum(prediction_window) / len(prediction_window)

        cv2.putText(frame, f"Status: {label}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 0, 255) if label == "Drowsy" else (0, 255, 0), 2)

        if len(prediction_window) == 30 and avg_drowsy > 0.6:
            cv2.putText(frame, "⚠ WAKE UP! Drowsiness Detected ⚠", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if not alarm_active:
                alarm_active = True
                threading.Thread(target=play_alarm, daemon=True).start()

        cv2.imshow("Drowsiness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


👁 EAR: 0.323
👁 EAR: 0.305
👁 EAR: 0.248
👁 EAR: 0.283
👁 EAR: 0.209
👁 EAR: 0.224
👁 EAR: 0.275
👁 EAR: 0.286
👁 EAR: 0.268
👁 EAR: 0.277
👁 EAR: 0.258
👁 EAR: 0.250
👁 EAR: 0.265
👁 EAR: 0.259
👁 EAR: 0.232
👁 EAR: 0.251
👁 EAR: 0.253
👁 EAR: 0.288
👁 EAR: 0.254
👁 EAR: 0.269
👁 EAR: 0.273
👁 EAR: 0.271
👁 EAR: 0.273
👁 EAR: 0.279
👁 EAR: 0.262
👁 EAR: 0.259
👁 EAR: 0.270
👁 EAR: 0.254
👁 EAR: 0.258
👁 EAR: 0.266
👁 EAR: 0.266
👁 EAR: 0.272
👁 EAR: 0.242
👁 EAR: 0.259
👁 EAR: 0.264
👁 EAR: 0.272
👁 EAR: 0.275
👁 EAR: 0.250
👁 EAR: 0.277
👁 EAR: 0.264
👁 EAR: 0.285
👁 EAR: 0.251
👁 EAR: 0.275
👁 EAR: 0.263
👁 EAR: 0.185
👁 EAR: 0.205
👁 EAR: 0.200
👁 EAR: 0.235
👁 EAR: 0.247
👁 EAR: 0.220
👁 EAR: 0.224
👁 EAR: 0.248
👁 EAR: 0.262
👁 EAR: 0.249
👁 EAR: 0.270
👁 EAR: 0.261
👁 EAR: 0.273
👁 EAR: 0.265
👁 EAR: 0.249
👁 EAR: 0.251
👁 EAR: 0.220
👁 EAR: 0.237
👁 EAR: 0.251
👁 EAR: 0.269
👁 EAR: 0.275
👁 EAR: 0.246
👁 EAR: 0.269
👁 EAR: 0.262
👁 EAR: 0.248
👁 EAR: 0.280
👁 EAR: 0.261
👁 EAR: 0.241
👁 EAR: 0.262
👁 EAR: 0.271
👁 EAR: 0.252
👁 EAR: 0.245
👁 EAR: 0.262

ERROR:playsound:
    Error 259 for command:
        play "C:\Users\blatw\OneDrive\Desktop\alarm.mp3.mp3" wait
    The driver cannot recognize the specified command parameter.
Exception in thread Thread-3 (play_alarm):
Traceback (most recent call last):
  File "C:\Users\blatw\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\blatw\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\blatw\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\blatw\AppData\Local\Temp\ipykernel_43720\201009557.py", line 22, in play_alarm
  File "C:\Users\blatw\AppData\Local\Programs\Python\Python311\Lib\site-packages\playsound.py", line 73, in _playsoundWin
    winCommand(u'play {}{}'.format(sound, ' wait' if block else ''))
  File "C:\Users\blatw\Ap

👁 EAR: 0.189
👁 EAR: 0.209
👁 EAR: 0.202
👁 EAR: 0.206
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.182
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.209
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.202
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.196
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.204
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.200
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.185
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.191
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.194
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.202
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.193
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.189
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.193
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.201
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.199
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.188
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.184
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.200
🤖 CNN Prediction: 0.00 -> Alert
👁 EAR: 0.215
👁 EAR: 0.228
👁 EAR: 0.220
👁 EAR: 0.232
👁 EAR: 0.213
👁 EAR: 0.231
👁 EAR: 0.214
👁 EAR: 0.224
👁 